In [119]:
import xarray as xr
import numpy as np
#import pandas as pd
import os
import glob
import copy as cp
import random
import pyproj as proj
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import normalize
from sklearn.model_selection import KFold
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn import cluster
import wpca
import pandas as pd
import pickle 
from matplotlib.cm import get_cmap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy.crs as ccrs
import cartopy as cr
import matplotlib.cm as cm
from scipy.stats.stats import pearsonr   
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from datetime import datetime

In [120]:
SUB_SETS = {'all':{'Subset_name':'/storage/nplanat/Glorys12_OP_journalier/NEW_RUN/Sub_samp_y_1993_-_2014_delta_1_100000_start_lon_0.nc',\
                  'init_year':1993,\
                  'final_year':2014,\
                   'delta_year':1,\
                  'N_particles':100000,\
                  'Subset_path':'/storage/nplanat/Glorys12_OP_journalier/NEW_RUN/'},\
           'weird_path_only':{'Subset_name':'/storage/nplanat/Glorys12_OP_journalier/WEIRD/Sub_samp_y_1996_-_1997_-delta_1_40000_start_lon_0.nc',\
                  'init_year':1996,\
                  'final_year':1997,\
                   'delta_year':1,\
                  'N_particles':40000,\
                  'Subset_path':'/storage/nplanat/Glorys12_OP_journalier/WEIRD/'},\
           'without_weird_path':{'Subset_name':None,\
                  'init_year':1993,\
                  'final_year':2014,\
                   'delta_year':1,\
                  'N_particles':100000,\
                  'Subset_path':'/storage/nplanat/Glorys12_OP_journalier/NO_WEIRD/'},\
           'all_short':{'Subset_name':'/storage/nplanat/Glorys12_OP_journalier/SHORT/Sub_samp_y_1993_-_2014_delta_1_100000_start_lon_0.nc',\
                  'init_year':1993,\
                  'final_year':2014,\
                   'delta_year':1,\
                  'N_particles':100000,\
                  'Subset_path':'/storage/nplanat/Glorys12_OP_journalier/SHORT/'}}

In [121]:
sel_subset = 'all_short'
load_previous_run = True
init_year = SUB_SETS[sel_subset]['init_year']
final_year = SUB_SETS[sel_subset]['final_year']
delta_year = SUB_SETS[sel_subset]['delta_year']
Subset_name = SUB_SETS[sel_subset]['Subset_name']
N_particles = SUB_SETS[sel_subset]['N_particles']
Subset_path = SUB_SETS[sel_subset]['Subset_path']

files = sorted(glob.glob('/storage/nplanat/Glorys12_OP_journalier/ADV_new_*'))
files_m = sorted(glob.glob('/storage/nplanat/Glorys12_OP_journalier/ADV_m_200_*'))
files_s = sorted(glob.glob('/storage/nplanat/Glorys12_OP_journalier/ADV_s_200_*'))
length_days = 450
path_save_prediction = Subset_path 


perctest = 0.1 
perctrain = 0.8
trans = True

#PCA 
n_components = 0.9999999

# Clustering
init = 'k-means++'
nmb_initialisations = 20  # number of initiatilisaton for the k-means++ 
max_iter = 500
tol = 5e-4
algorithm = 'full'
verbose = 0 
sample_weight = None
n_split = 20  # number of iterations for convergence
n_clusters = 20 #number of clusters

# Load Classif

In [122]:
#load PCA : 
with open(Subset_path + 'pca.pkl', 'rb') as pickle_file:
    pca = pickle.load(pickle_file)
#load clustering : 
Traj_centroids = np.load(Subset_path+'Traj_centroids.npy', allow_pickle = True)
# load previous runs :
Labels_valid = np.load(path_save_prediction+'Labels_valid.npy')
Labels_test = np.load(path_save_prediction+'Labels_test.npy')
lats_test = np.load(path_save_prediction+'lats_test.npy')
lons_test = np.load(path_save_prediction+'lons_test.npy')
lats_train = np.load(path_save_prediction+'lats_train.npy')
lons_train = np.load(path_save_prediction+'lons_train.npy')
lats_valid = np.load(path_save_prediction+'lats_valid.npy')
lons_valid = np.load(path_save_prediction+'lons_valid.npy')

# Apply to all data

In [123]:
def extract_all_per_year(yr, length_days, all_filles) :
    file = all_filles[np.where(np.array([int(all_filles[i][-7:-3]) for i in range(len(all_filles))]) == yr)[0][0]]
    lats, lons, temps, sal, date, z = extract(file, length_days)    
    return lats, lons, temps, sal, date, z

def apply_to_all_data(Subset_path, init_year,final_year, length_days, files, t, sp, trans, Traj_centroids):

    for yr in range(init_year,final_year):
        print('Year', yr)
        #lats_all, lons_all, temps_all, sals_all, date_all = extract_all_per_year(yr, length_days, files)
        lats_all = np.load('/storage/nplanat/Glorys12_OP_journalier/lats_lons/lats_%i'%yr+'.npy', allow_pickle = True)
        lons_all = np.load('/storage/nplanat/Glorys12_OP_journalier/lats_lons/lons_%i'%yr+'.npy', allow_pickle = True)
        labels_all = []
        for sep in range(0,lats_all.shape[0],sp) :
            print(sep,'/',lats_all.shape[0])
            lats_all_l = lats_all[sep:sep+sp,:length_days] ; lons_all_l = lons_all[sep:sep+sp,:length_days]
            
            X_lats_lons_to_classify = Part_1_pre_processing_one_sort(trans, lats_all_l, lons_all_l)
            X_reduced_to_classify = pca.transform(X_lats_lons_to_classify)
            labels = predict(X_reduced_to_classify, Traj_centroids)
            labels_all.extend(labels)
        np.save(Subset_path+'labels_data_%i'%yr, labels_all)
        #np.save(Subset_path+'/'+'lats_data_%i'%yr, lats_all)
        #np.save(Subset_path+'/'+'lons_data_%i'%yr, lons_all)
    return None

In [124]:
def split_sets(lats_all, lons_all, temps_all, sals_all, perctest, perctrain):
    
    s0, s1 = lats_all.shape
    data = np.concatenate((lats_all,lons_all), axis=1)
    
    random.seed(4)
    random.shuffle(data)
    random.seed(4)
    random.shuffle(temps_all)
    random.seed(4)
    random.shuffle(sals_all)

    lats_test = data[:int(perctest*s0), 0:s1]
    lons_test = data[:int(perctest*s0), s1:]

    lats_train = data[int(perctest*s0):int(perctest*s0)+int(perctrain*s0), 0:s1]
    lons_train = data[int(perctest*s0):int(perctest*s0)+int(perctrain*s0), s1:]

    lats_valid = data[int(perctest*s0)+int(perctrain*s0):, 0:s1]
    lons_valid = data[int(perctest*s0)+int(perctrain*s0):, s1:]

    temps_test = temps_all[:int(perctest*s0),:]
    sals_test = sals_all[:int(perctest*s0),:]
    
    temps_train =temps_all[int(perctest*s0):int(perctest*s0)+int(perctrain*s0),:]
    sals_train = sals_all[int(perctest*s0):int(perctest*s0)+int(perctrain*s0),:]
    
    temps_valid = temps_all[int(perctest*s0)+int(perctrain*s0):,:]
    sals_valid = sals_all[int(perctest*s0)+int(perctrain*s0):,:]
    
    return lats_test, lons_test, lats_train, lons_train, lats_valid, lons_valid, temps_train, temps_test, temps_valid, sals_train, sals_test, sals_valid

In [125]:
def lons_in_interval(start_lon, lons):
    #this should shift all longitudes so it lies in start_lon + 360. 
    if start_lon<0 : 
        print('start_lons should be in [0-360[')
        return None
    else:
        lons_shifted = lons
        while np.any(lons_shifted<0):
            lons_shifted[lons_shifted<0] = lons_shifted[lons_shifted<0]+360
        lons_2 = cp.copy(lons_shifted)
        lons_2[lons_shifted>=start_lon] = lons_shifted[lons_shifted>=start_lon] - start_lon
        lons_2[lons_shifted<start_lon] = lons_shifted[lons_shifted<start_lon] - start_lon +360
        return cp.copy(lons_2)

In [126]:
def extract(path, length_days) :
    ds = xr.open_dataset(path)
    ds = remove_short(ds)
    lats = ds.lat[:,0:length_days].values
    lons_i = ds.lon[:,0:length_days].values
    lons = lons_in_interval(0, lons_i) # by default start_lon = 360
    temps = ds.temperature[:,0:length_days].values
    sal = ds.salinity[:,0:length_days].values
    z = ds.z[:,0:length_days].values
    date = ds.time[:,0].values
    return lats, lons, temps, sal, date, z

In [127]:
def translate(lat, lon):
    return lat-np.repeat(lat[:,0][:, np.newaxis], lat.shape[1], axis=1), lon-np.repeat(lon[:,0][:, np.newaxis], lon.shape[1], axis=1)

In [128]:
def Part_1_pre_processing_one_sort(t, la, lo):
    if t == True :
        lats_train_pp, lons_train_pp = translate(la, lo)

    # Reshape
    X_lats_lons_train = np.concatenate((lats_train_pp, lons_train_pp), axis = 1) #concatenation of features to make it in the correct shape
    X_lats_lons_train[np.isnan(X_lats_lons_train)] = 0 
   
    return X_lats_lons_train

In [129]:
def Part_1_pre_processing(Dataset_path, length_days, perctest, perctrain, t):
    lats_all,lons_all, temps_all, sals_all, date_all, depth_all = extract(Dataset_path, length_days)
    lats_test, lons_test, lats_train, lons_train, lats_valid, lons_valid, temps_train, temps_test, temps_valid, sals_train, sals_test, sals_valid = split_sets(lats_all, lons_all,temps_all, sals_all, perctest, perctrain)

    print('Pre_processing')
    # PRE-PROCESSING
    X_lats_lons_train = Part_1_pre_processing_one_sort(t, lats_train, lons_train)
    X_lats_lons_test = Part_1_pre_processing_one_sort(t, lats_test, lons_test)
    X_lats_lons_valid = Part_1_pre_processing_one_sort(t, lats_valid, lons_valid)
    
    return X_lats_lons_valid, X_lats_lons_test, X_lats_lons_train, lats_test, lons_test, lats_train, lons_train, lats_valid, lons_valid,

In [130]:
def predict(X_reduced_val, X_centers):
    
    X_reduced_val2 = np.repeat(X_reduced_val[:,:,np.newaxis], X_centers.shape[0], axis = 2)
    X_centers2 = np.repeat(np.transpose(X_centers)[np.newaxis,:,:], X_reduced_val.shape[0], axis = 0)
    
    return np.argmin(np.linalg.norm(X_reduced_val2-X_centers2, axis=1), axis = 1)

In [131]:
def apply_to_all_data_m(Subset_path, init_year,final_year, length_days, files, t, sp, trans,Traj_centroids ):

    for yr in range(init_year,final_year):
        print('Year', yr)
        #lats_all, lons_all, temps_all, sals_all, date_all = extract_all_per_year(yr, length_days, files)
        lats_all = np.load('/storage/nplanat/Glorys12_OP_journalier/lats_lons/m_lats_%i'%yr+'.npy', allow_pickle = True)
        lons_all = np.load('/storage/nplanat/Glorys12_OP_journalier/lats_lons/m_lons_%i'%yr+'.npy', allow_pickle = True)
        labels_all = []
        for sep in range(0,lats_all.shape[0],sp) :
            print(sep,'/',lats_all.shape[0])
            lats_all_l = lats_all[sep:sep+sp,:length_days] ; lons_all_l = lons_all[sep:sep+sp,:length_days]
            
            X_lats_lons_to_classify = Part_1_pre_processing_one_sort(trans, lats_all_l, lons_all_l)
            X_reduced_to_classify = pca.transform(X_lats_lons_to_classify)
            labels = predict(X_reduced_to_classify, Traj_centroids)
            labels_all.extend(labels)
        np.save(Subset_path+'m_labels_data_%i'%yr, labels_all)
        #np.save(Subset_path+'/'+'lats_data_%i'%yr, lats_all)
        #np.save(Subset_path+'/'+'lons_data_%i'%yr, lons_all)
    return None

In [132]:
def apply_to_all_data_s(Subset_path, init_year,final_year, length_days, files, t, sp, trans,Traj_centroids ):

    for yr in range(init_year,final_year):
        print('Year', yr)
        #lats_all, lons_all, temps_all, sals_all, date_all = extract_all_per_year(yr, length_days, files)
        lats_all = np.load('/storage/nplanat/Glorys12_OP_journalier/lats_lons/s_lats_%i'%yr+'.npy', allow_pickle = True)
        lons_all = np.load('/storage/nplanat/Glorys12_OP_journalier/lats_lons/s_lons_%i'%yr+'.npy', allow_pickle = True)
        labels_all = []
        for sep in range(0,lats_all.shape[0],sp) :
            print(sep,'/',lats_all.shape[0])
            lats_all_l = lats_all[sep:sep+sp,:length_days] ; lons_all_l = lons_all[sep:sep+sp,:length_days]
            
            X_lats_lons_to_classify = Part_1_pre_processing_one_sort(trans, lats_all_l, lons_all_l)
            X_reduced_to_classify = pca.transform(X_lats_lons_to_classify)
            labels = predict(X_reduced_to_classify, Traj_centroids)
            labels_all.extend(labels)
        np.save(Subset_path+'s_labels_data_%i'%yr, labels_all)
        #np.save(Subset_path+'/'+'lats_data_%i'%yr, lats_all)
        #np.save(Subset_path+'/'+'lons_data_%i'%yr, lons_all)
    return None

In [133]:
apply_to_all_data(path_save_prediction, 1993,2014, length_days, files, trans, sp = 5000,  trans = trans, Traj_centroids = Traj_centroids)

Year 1993
0 / 23129
5000 / 23129
10000 / 23129
15000 / 23129
20000 / 23129
Year 1994
0 / 23154
5000 / 23154
10000 / 23154
15000 / 23154
20000 / 23154
Year 1995
0 / 24140
5000 / 24140
10000 / 24140
15000 / 24140
20000 / 24140
Year 1996
0 / 23589
5000 / 23589
10000 / 23589
15000 / 23589
20000 / 23589
Year 1997
0 / 23424
5000 / 23424
10000 / 23424
15000 / 23424
20000 / 23424
Year 1998
0 / 23578
5000 / 23578
10000 / 23578
15000 / 23578
20000 / 23578
Year 1999
0 / 23619
5000 / 23619
10000 / 23619
15000 / 23619
20000 / 23619
Year 2000
0 / 23119
5000 / 23119
10000 / 23119
15000 / 23119
20000 / 23119
Year 2001
0 / 23331
5000 / 23331
10000 / 23331
15000 / 23331
20000 / 23331
Year 2002
0 / 23207
5000 / 23207
10000 / 23207
15000 / 23207
20000 / 23207
Year 2003
0 / 22522
5000 / 22522
10000 / 22522
15000 / 22522
20000 / 22522
Year 2004
0 / 22509
5000 / 22509
10000 / 22509
15000 / 22509
20000 / 22509
Year 2005
0 / 22593
5000 / 22593
10000 / 22593
15000 / 22593
20000 / 22593
Year 2006
0 / 23610
5000 

In [135]:
apply_to_all_data_m(path_save_prediction, 1993,2014, length_days, files_m, trans, sp = 5000,  trans = trans, Traj_centroids = Traj_centroids)

Year 1993
0 / 24562
5000 / 24562
10000 / 24562
15000 / 24562
20000 / 24562
Year 1994
0 / 24339
5000 / 24339
10000 / 24339
15000 / 24339
20000 / 24339
Year 1995
0 / 24475
5000 / 24475
10000 / 24475
15000 / 24475
20000 / 24475
Year 1996
0 / 24456
5000 / 24456
10000 / 24456
15000 / 24456
20000 / 24456
Year 1997
0 / 24417
5000 / 24417
10000 / 24417
15000 / 24417
20000 / 24417
Year 1998
0 / 24467
5000 / 24467
10000 / 24467
15000 / 24467
20000 / 24467
Year 1999
0 / 24496
5000 / 24496
10000 / 24496
15000 / 24496
20000 / 24496
Year 2000
0 / 24485
5000 / 24485
10000 / 24485
15000 / 24485
20000 / 24485
Year 2001
0 / 24489
5000 / 24489
10000 / 24489
15000 / 24489
20000 / 24489
Year 2002
0 / 24189
5000 / 24189
10000 / 24189
15000 / 24189
20000 / 24189
Year 2003
0 / 24112
5000 / 24112
10000 / 24112
15000 / 24112
20000 / 24112
Year 2004
0 / 24503
5000 / 24503
10000 / 24503
15000 / 24503
20000 / 24503
Year 2005
0 / 24436
5000 / 24436
10000 / 24436
15000 / 24436
20000 / 24436
Year 2006
0 / 24525
5000 

In [137]:
apply_to_all_data_s(path_save_prediction, 1993, 2014, length_days, files_s, trans, sp = 5000,  trans = trans, Traj_centroids = Traj_centroids)

Year 1993
0 / 24657
5000 / 24657
10000 / 24657
15000 / 24657
20000 / 24657
Year 1994
0 / 24597
5000 / 24597
10000 / 24597
15000 / 24597
20000 / 24597
Year 1995
0 / 24576
5000 / 24576
10000 / 24576
15000 / 24576
20000 / 24576
Year 1996
0 / 24491
5000 / 24491
10000 / 24491
15000 / 24491
20000 / 24491
Year 1997
0 / 24503
5000 / 24503
10000 / 24503
15000 / 24503
20000 / 24503
Year 1998
0 / 24577
5000 / 24577
10000 / 24577
15000 / 24577
20000 / 24577
Year 1999
0 / 24583
5000 / 24583
10000 / 24583
15000 / 24583
20000 / 24583
Year 2000
0 / 24620
5000 / 24620
10000 / 24620
15000 / 24620
20000 / 24620
Year 2001
0 / 24641
5000 / 24641
10000 / 24641
15000 / 24641
20000 / 24641
Year 2002
0 / 24587
5000 / 24587
10000 / 24587
15000 / 24587
20000 / 24587
Year 2003
0 / 24475
5000 / 24475
10000 / 24475
15000 / 24475
20000 / 24475
Year 2004
0 / 24569
5000 / 24569
10000 / 24569
15000 / 24569
20000 / 24569
Year 2005
0 / 24631
5000 / 24631
10000 / 24631
15000 / 24631
20000 / 24631
Year 2006
0 / 24563
5000 